# ANOVA  - Lab

## Introduction

In this lab, you'll get some brief practice generating an ANOVA table (AOV) and interpreting its output. You'll also perform some investigations to compare the method to the t-tests you previously employed to conduct hypothesis testing.

## Objectives

In this lab you will: 

- Use ANOVA for testing multiple pairwise comparisons 
- Interpret results of an ANOVA and compare them to a t-test

## Load the data

Start by loading in the data stored in the file `'ToothGrowth.csv'`: 

In [5]:
# Your code here
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats
df = pd.read_csv('ToothGrowth.csv')
df.head()

,len,supp,dose
0,4.2,VC,0.5
1,11.5,VC,0.5
2,7.3,VC,0.5
3,5.8,VC,0.5
4,6.4,VC,0.5


## Generate the ANOVA table

Now generate an ANOVA table in order to analyze the influence of the medication and dosage:  

In [3]:
# Your code here
formula = 'len ~  C(supp) + C(dose)'
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

               sum_sq    df          F        PR(>F)
C(supp)    205.350000   1.0  14.016638  4.292793e-04
C(dose)   2426.434333   2.0  82.810935  1.871163e-17
Residual   820.425000  56.0        NaN           NaN


## Interpret the output

Make a brief comment regarding the statistics and the effect of supplement and dosage on tooth length: 

# Your comment here
Both supp and dose PR(>F) values are less than 0.05, making both important, with dose having a greater effect

## Compare to t-tests

Now that you've had a chance to generate an ANOVA table, its interesting to compare the results to those from the t-tests you were working with earlier. With that, start by breaking the data into two samples: those given the OJ supplement, and those given the VC supplement. Afterward, you'll conduct a t-test to compare the tooth length of these two different samples: 

In [4]:
# Your code here
oj = df[df.supp=='OJ']['len']
vc = df[df.supp=='VC']['len']

Now run a t-test between these two groups and print the associated two-sided p-value: 

In [6]:
# Calculate the 2-sided p-value for a t-test comparing the two supplement groups
stats.ttest_ind(oj, vc, equal_var=False)

Ttest_indResult(statistic=1.91526826869527, pvalue=0.06063450788093387)

## A 2-Category ANOVA F-test is equivalent to a 2-tailed t-test!

Now, recalculate an ANOVA F-test with only the supplement variable. An ANOVA F-test between two categories is the same as performing a 2-tailed t-test! So, the p-value in the table should be identical to your calculation above.

> Note: there may be a small fractional difference (>0.001) between the two values due to a rounding error between implementations. 

In [7]:
# Your code here; conduct an ANOVA F-test of the oj and vc supplement groups.
# Compare the p-value to that of the t-test above. 
# They should match (there may be a tiny fractional difference due to rounding errors in varying implementations)
formula2 = 'len ~ C(supp)'
lm2 = ols(formula2, df).fit()
table2 = sm.stats.anova_lm(lm2, typ=2)
print(table2)

               sum_sq    df         F    PR(>F)
C(supp)    205.350000   1.0  3.668253  0.060393
Residual  3246.859333  58.0       NaN       NaN


## Run multiple t-tests

While the 2-category ANOVA test is identical to a 2-tailed t-test, performing multiple t-tests leads to the multiple comparisons problem. To investigate this, look at the various sample groups you could create from the 2 features: 

In [8]:
for group in df.groupby(['supp', 'dose'])['len']:
    group_name = group[0]
    data = group[1]
    print(group_name)

('OJ', 0.5)
('OJ', 1.0)
('OJ', 2.0)
('VC', 0.5)
('VC', 1.0)
('VC', 2.0)


While bad practice, examine the effects of calculating multiple t-tests with the various combinations of these. To do this, generate all combinations of the above groups. For each pairwise combination, calculate the p-value of a 2-sided t-test. Print the group combinations and their associated p-value for the two-sided t-test.

In [10]:
# Your code here; reuse your t-test code above to calculate the p-value for a 2-sided t-test
# for all combinations of the supplement-dose groups listed above. 
# (Since there isn't a control group, compare each group to every other group.)
from itertools import combinations
groups = []
for group in df.groupby(['supp', 'dose'])['len']:
    groups.append(group[0])
combin = combinations(groups, 2)
for c in combin:
    supp_1 = c[0][0]
    dose_1 = c[0][1]
    supp_2 = c[1][0]
    dose_2 = c[1][1]
    x1 = df[(df.supp==supp_1) & (df.dose==dose_1)]['len']
    x2 = df[(df.supp==supp_2) & (df.dose==dose_2)]['len']
    pval = stats.ttest_ind(x1, x2, equal_var=False)
    print(c, pval)

(('OJ', 0.5), ('OJ', 1.0)) Ttest_indResult(statistic=-5.048634657337134, pvalue=8.784919055161479e-05)
(('OJ', 0.5), ('OJ', 2.0)) Ttest_indResult(statistic=-7.81702129580037, pvalue=1.3237838776972294e-06)
(('OJ', 0.5), ('VC', 0.5)) Ttest_indResult(statistic=3.1697327836708125, pvalue=0.006358606764096813)
(('OJ', 0.5), ('VC', 1.0)) Ttest_indResult(statistic=-2.1863610837918044, pvalue=0.04601033257637553)
(('OJ', 0.5), ('VC', 2.0)) Ttest_indResult(statistic=-6.232482137515355, pvalue=7.196253524006043e-06)
(('OJ', 1.0), ('OJ', 2.0)) Ttest_indResult(statistic=-2.2477612380279295, pvalue=0.039195142046244004)
(('OJ', 1.0), ('VC', 0.5)) Ttest_indResult(statistic=9.740112783652796, pvalue=3.6552067303259103e-08)
(('OJ', 1.0), ('VC', 1.0)) Ttest_indResult(statistic=4.03276963371935, pvalue=0.001038375872299884)
(('OJ', 1.0), ('VC', 2.0)) Ttest_indResult(statistic=-1.757441811684785, pvalue=0.09652612338267014)
(('OJ', 2.0), ('VC', 0.5)) Ttest_indResult(statistic=14.966531872499512, pvalue=

## Summary

In this lesson, you implemented the ANOVA technique to generalize testing methods to multiple groups and factors.